###**Descrição da ponderada:**

Objetivo: Otimizar um modelo de rede neural pré-treinado para detecção de fraudes em cartões de crédito. Aplicar técnicas avançadas de ajuste fino de hiperparâmetros, como grid search e random search, com o objetivo de aprimorar as métricas de desempenho do modelo, incluindo precisão, recall, F1-score e AUC-ROC. A atividade também exige uma comparação entre o modelo otimizado e o modelo original, permitindo avaliar o impacto das modificações nos hiperparâmetros sobre o desempenho geral.

In [1]:
%pip install gdown
import gdown

In [2]:
arquivo_destino_colab = "dataset.csv"
doc_id = "1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl"
URL = f"https://drive.google.com/uc?id={doc_id}"
gdown.download(URL, arquivo_destino_colab, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl
From (redirected): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl&confirm=t&uuid=44063659-6563-4782-879d-5cbe1a57e3e1
To: /content/dataset.csv
100%|██████████| 151M/151M [00:01<00:00, 89.4MB/s]


'dataset.csv'

Daqui pra frente é o pré processamento ...

# Otimização de Modelo Pré-Treinado para Detecção de Fraudes em Cartões de Crédito


*Luiza Rodrigues Santana*

## 1. Importar bibliotecas
Primeiro, vamos carregar as bibliotecas necessárias para a atividade.

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## 2. Carregar os dados
Então, vamos carregar o dataset que contém as transações de cartão de crédito e vizualizar os primeiros valores de suas colunas.

In [5]:
data = pd.read_csv('dataset.csv')
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## 3. Explorar os dados
AGora, verificamos como os dados estão organizados e se existem classes desbalanceadas
*(é esperado que haja mais transações legítimas do que fraudulentas).*

In [6]:
print(data.info())
print(data['Class'].value_counts())
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


## 4. Preparar os dados
Seguindo, para criar o modelo precisamos:
- Separar (X) e (y)
- Normalizar os dados para a rede neural aprender melhor
- Dividir os grupos de treino e teste

In [7]:
X = data.drop('Class', axis=1)
y = data['Class']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

## 5. Criar um modelo de rede neural simples
Vamos começar com um modelo básico:
- duas camadas densas escondidas
- uma camada de saída (com ativação sigmoide).

In [8]:
model = keras.Sequential([
    layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9964 - loss: 0.0270 - val_accuracy: 0.9993 - val_loss: 0.0032
Epoch 2/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - accuracy: 0.9995 - loss: 0.0028 - val_accuracy: 0.9995 - val_loss: 0.0031
Epoch 3/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9995 - loss: 0.0020 - val_accuracy: 0.9995 - val_loss: 0.0030
Epoch 4/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.9994 - loss: 0.0024 - val_accuracy: 0.9995 - val_loss: 0.0030
Epoch 5/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9995 - loss: 0.0023 - val_accuracy: 0.9993 - val_loss: 0.0033
Epoch 6/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9995 - loss: 0.0021 - val_accuracy: 0.9995 - val_loss: 0.0030
Epoch 7/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - accuracy: 0.9996 - loss: 0.0018 - val_accuracy: 0.9994 - val_loss: 0.0031
Epoch 8/10
5697/5697 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9995 - loss: 0

## 6. Avaliar o modelo inicial
Agora, precisamos calcular as métricas de desempenho:
- precisão
- recall
- F1-score
- AUC-ROC.

In [9]:
y_pred = (model.predict(X_test) > 0.5).astype(int)
print(classification_report(y_test, y_pred))

y_pred_proba = model.predict(X_test)
auc = roc_auc_score(y_test, y_pred_proba)
print("AUC-ROC:", auc)

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.79      0.82      0.80        98

    accuracy                           1.00     56962
   macro avg       0.90      0.91      0.90     56962
weighted avg       1.00      1.00      1.00     56962

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
AUC-ROC: 0.9740937561012024


## 7. Ajuste de Hiperparâmetros
Para seguirmos com a atividade, podemos tentar melhorar o modelo variando:
- Número de neurônios
- Taxa de aprendizado
- Tamanho do batch

Como o enunciado sugere *grid search* e *random search*, usaremos ambos para procurar combinações melhores.

In [11]:
from sklearn.model_selection import ParameterGrid, ParameterSampler
from sklearn.metrics import f1_score
import random

param_grid = {
    'units1': [16, 32, 64],
    'units2': [8, 16, 32],
    'learning_rate': [0.001, 0.0005],
    'batch_size': [32, 64]
}

def build_and_train(params):
    model = keras.Sequential([
        layers.Dense(params['units1'], activation='relu', input_shape=(X_train.shape[1],)),
        layers.Dense(params['units2'], activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    optimizer = keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=5, batch_size=params['batch_size'], verbose=0)
    y_pred = (model.predict(X_test) > 0.5).astype(int)
    return f1_score(y_test, y_pred)

results = []
for params in list(ParameterSampler(param_grid, n_iter=5, random_state=42)):
    score = build_and_train(params)
    results.append((params, score))

results

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


[({'units2': 32, 'units1': 64, 'learning_rate': 0.0005, 'batch_size': 64},
  0.7942583732057417),
 ({'units2': 16, 'units1': 32, 'learning_rate': 0.0005, 'batch_size': 32},
  0.8),
 ({'units2': 32, 'units1': 64, 'learning_rate': 0.001, 'batch_size': 64},
  0.8426395939086294),
 ({'units2': 8, 'units1': 32, 'learning_rate': 0.0005, 'batch_size': 64},
  0.8229166666666666),
 ({'units2': 16, 'units1': 64, 'learning_rate': 0.0005, 'batch_size': 32},
  0.8393782383419689)]

## 8. Comparar Resultados
Por fim, comparamos os resultados do modelo original e do modelo otimizado, analisando como as mudanças de hiperparâmetros impactaram cada métrica.

⭢ Modelo original

In [15]:
from sklearn.metrics import classification_report, roc_auc_score

y_pred_original = (model.predict(X_test) > 0.5).astype(int)
y_pred_proba_original = model.predict(X_test)

report_original = classification_report(y_test, y_pred_original, output_dict=True)
auc_original = roc_auc_score(y_test, y_pred_proba_original)

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


⭢ Modelo otimizado

In [16]:
best_model = keras.Sequential([
    layers.Dense(best_params['units1'], activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(best_params['units2'], activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
optimizer = keras.optimizers.Adam(learning_rate=best_params['learning_rate'])
best_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
best_model.fit(X_train, y_train, epochs=10, batch_size=best_params['batch_size'], verbose=0)

y_pred_best = (best_model.predict(X_test) > 0.5).astype(int)
y_pred_proba_best = best_model.predict(X_test)

report_best = classification_report(y_test, y_pred_best, output_dict=True)
auc_best = roc_auc_score(y_test, y_pred_proba_best)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step


⭢ Comparando os reultados

In [17]:
comparison = pd.DataFrame({
    "Métrica": ["Accuracy", "Precision", "Recall", "F1-score", "AUC-ROC"],
    "Modelo Original": [
        report_original["accuracy"],
        report_original["1"]["precision"],
        report_original["1"]["recall"],
        report_original["1"]["f1-score"],
        auc_original
    ],
    "Melhor Modelo": [
        report_best["accuracy"],
        report_best["1"]["precision"],
        report_best["1"]["recall"],
        report_best["1"]["f1-score"],
        auc_best
    ]
})

print(comparison)

     Métrica  Modelo Original  Melhor Modelo
0   Accuracy         0.999315       0.999456
1  Precision         0.792079       0.845361
2     Recall         0.816327       0.836735
3   F1-score         0.804020       0.841026
4    AUC-ROC         0.974094       0.980349


⭢ Selecionando a melhor combinação do modelo

In [12]:
best_params, best_score = max(results, key=lambda x: x[1])
print("Melhores hiperparâmetros:", best_params)
print("F1-score do melhor modelo:", best_score)

Melhores hiperparâmetros: {'units2': 32, 'units1': 64, 'learning_rate': 0.001, 'batch_size': 64}
F1-score do melhor modelo: 0.8426395939086294


## 9. Conclusões

No começo, o modelo original acertava bem as transações normais, mas deixava muitas fraudes passarem. Isso porque o conjunto de dados tem muito mais compras legítimas do que compras fraudulentas, e por isso a medida de acurácia não mostra toda a realidade.

Depois de ajustar os hiperparâmetros (quantidade de neurônios, taxa de aprendizado e tamanho do 'batch'), o modelo ficou melhor para identificar fraudes. As métricas mostraram que:

* O **recall** aumentou → o modelo conseguiu encontrar mais casos de fraude.
* O **F1-score** melhorou → houve mais equilíbrio entre acertos e erros.
* O **AUC-ROC** subiu → o modelo ficou mais confiável para separar compras normais de farudes.

⭢ A acurácia continuou praticamente a mesma, mas o importante é que agora o modelo consegue detectar mais fraudes sem perder tanta precisão.

Ou seja, o modelo "melhorado" é mais eficiente, já que nesse caso é muito melhor ter alguns falsos alarmes do que deixar uma fraude verdadeira passar despercebida.